# 70-色彩追踪（Color Tracking）

在HSV色彩空间内对图像创建一个只有蓝色部分值为255的图像。

色彩追踪是提取特定颜色的区域的方法。

然而，由于在 RGB 色彩空间内颜色有$256^3$种，因此十分困难（或者说手动提取相当困难），因此进行 HSV 变换。

HSV 变换在问题5中提到过，是将 RGB 变换到色相（Hue）、饱和度（Saturation）、明度（Value）的方法。

饱和度越小越白，饱和度越大颜色越浓烈，$0\leq S\leq 1$；

明度数值越高越接近白色，数值越低越接近黑色（$0\leq V\leq 1$）；

色相：将颜色使用0到360度表示，具体色相与数值按下表对应：

红	黄	绿	青色	蓝色	品红	红

0°	60°	120°	180°	240°	300°	360°

也就是说，为了追踪蓝色，可以在进行 HSV 转换后提取其中$180\leq H\leq 260$的位置，将其变为$255$。

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread('../picture/chans.png').astype(np.float32)

In [3]:
def BGR2HSV(_img):
    
    img = _img.copy() / 255.

    hsv = np.zeros_like(img, dtype=np.float32)

    max_v = np.max(img, axis=2).copy()
    min_v = np.min(img, axis=2).copy()
    min_arg = np.argmin(img, axis=2)

    # H
    hsv[..., 0][np.where(max_v == min_v)]= 0
    ## if min == B
    ind = np.where(min_arg == 0)
    hsv[..., 0][ind] = 60 * (img[..., 1][ind] - img[..., 2][ind]) / (max_v[ind] - min_v[ind]) + 60
    ## if min == R
    ind = np.where(min_arg == 2)
    hsv[..., 0][ind] = 60 * (img[..., 0][ind] - img[..., 1][ind]) / (max_v[ind] - min_v[ind]) + 180
    ## if min == G
    ind = np.where(min_arg == 1)
    hsv[..., 0][ind] = 60 * (img[..., 2][ind] - img[..., 0][ind]) / (max_v[ind] - min_v[ind]) + 300
    
    # S
    hsv[..., 1] = max_v.copy() - min_v.copy()
    
    # V
    hsv[..., 2] = max_v.copy()
    
    return hsv

In [4]:
def get_mask(hsv):
    
    mask = np.zeros_like(hsv[..., 0])
    #mask[np.where((hsv > 180) & (hsv[0] < 260))] = 255
    mask[np.logical_and((hsv[..., 0] > 180), (hsv[..., 0] < 260))] = 255

    return mask

In [5]:
hsv = BGR2HSV(img)

mask = get_mask(hsv)

out = mask.astype(np.uint8)

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


In [7]:
cv2.imwrite("../picture/result70_Color Tracking.jpg", out)
cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 71-掩膜（Masking）

使用HSV对imori.jpg进行掩膜处理，只让蓝色的地方变黑。

像这样通过使用黑白二值图像将对应于黑色部分的原始图像的像素改变为黑色的操作被称为掩膜。

要提取蓝色部分，请先创建这样的二进制图像，使得HSV色彩空间中$180\leq H\leq 260$的位置的像素值设为1，并将其0和1反转之后与原始图像相乘。

这使得可以在某种程度上将（从背景上）分离出来。

In [22]:
def get_masking(hsv):
    
    mask = np.zeros_like(hsv[..., 0])
    #mask[np.where((hsv > 180) & (hsv[0] < 260))] = 255
    mask[np.logical_and((hsv[..., 0] > 180), (hsv[..., 0] < 260))] = 1

    return mask

In [18]:
def masking(img, mask):
    
    mask = 1 - mask
    out = img.copy()
    
    # mask [h, w] -> [h, w, channel]
    mask = np.tile(mask, [3, 1, 1]).transpose([1, 2, 0])
    out *= mask

    return out

In [23]:
mask = get_masking(hsv)

out = masking(img, mask)

out = out.astype(np.uint8)

G:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
G:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """


In [24]:
cv2.imwrite("../picture/result71_masking.jpg", out)
cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 72-掩膜（色彩追踪（Color Tracking）+形态学处理）

在问题七十一中掩膜并不是十分精细。

因此，可以通过对掩膜图像应用N = 5闭运算（问题五十）和开运算（问题四十九），以使掩膜图像准确。

In [26]:
def Erode(img, Erode_time=1):
    
    H, W = img.shape
    out = img.copy()

    # kernel
    MF = np.array(((0, 1, 0), (1, 0, 1), (0, 1, 0)), dtype=np.int)

    for i in range(Erode_time):
        tmp = np.pad(out, (1, 1), 'edge')
        # erode
        for y in range(1, H + 1):
            for x in range(1, W + 1):
                if np.sum(MF * tmp[y - 1 : y + 2 , x - 1 : x + 2]) < 1 * 4:
                    out[y - 1, x - 1] = 0

    return out

In [27]:
def Dilate(img, Dil_time=1):

    H, W = img.shape
    
    # kernel
    MF = np.array(((0, 1, 0), (1, 0, 1), (0, 1, 0)), dtype=np.int)
    
    # each dilate time
    out = img.copy()
    for i in range(Dil_time):
        tmp = np.pad(out, (1, 1), 'edge')
        for y in range(1, H + 1):
            for x in range(1, W + 1):
                if np.sum(MF * tmp[y - 1 : y + 2, x - 1 : x + 2]) >= 1:
                    out[y - 1, x - 1] = 1

    return out

In [28]:
def Morphology_Opening(img, time=1):
    
    out = Erode(img, Erode_time=time)
    out = Dilate(out, Dil_time=time)
    
    return out

In [29]:
def Morphology_Closing(img, time=1):
    
    out = Dilate(img, Dil_time=time)
    out = Erode(out, Erode_time=time)
    
    return out

In [30]:
mask = Morphology_Closing(mask, time=5)

mask = Morphology_Opening(mask, time=5)

In [31]:
out = masking(img, mask)

out = out.astype(np.uint8)

In [ ]:
cv2.imwrite("../picture/result71_masking.jpg", out)
cv2.imshow("result", out)
cv2.waitKey(0)
cv2.destroyAllWindows()